## Loading the dependencies

In [3]:
import numpy as np
import cv2 as cv
from tqdm import tqdm

## Declaring global constants

In [4]:
CALIBRATION_VIDEO_PATH = '../data/calibration.mp4'
CAM_MATRIX_PATH  = '../processed/K.npy'
DISTORTION_PATH  = '../processed/dist.npy'
CHECKERBOARD_WIDTH   = 9
CHECKERBOARD_HEIGHT  = 6
TERMINATION_CRITERIA = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

## Performing the analysis

In [5]:
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# Coordinates of corners in the real world space
objp = np.zeros((CHECKERBOARD_WIDTH * CHECKERBOARD_HEIGHT, 3), np.float32)
objp[:,:2] = np.mgrid[0:CHECKERBOARD_WIDTH, 0:CHECKERBOARD_HEIGHT].T.reshape(-1,2)

### 1. Find the mappings

For each not skipped frame, we detect the chessboard corners using _findChessboardCorners_. We then save the mappings between corner projections and corner positions in 3d world space

In [19]:
cap = cv.VideoCapture(CALIBRATION_VIDEO_PATH)

# We will skip some frames in order to avoid calibration on similar frames
# and speed up the calibration process
SKIP_FRAMES  = 10
skip_counter = SKIP_FRAMES

camera_matrix = None

# Recall statistics
inspected = 0
detected  = 0

while cap.isOpened():
    ret, frame = cap.read()


    if not ret:
        break
    
    if skip_counter == 0:
        skip_counter = SKIP_FRAMES
        inspected += 1
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        if (camera_matrix == None):    
            camera_matrix = gray.shape[::-1]

        ret, corners = cv.findChessboardCorners(
            gray,
            (CHECKERBOARD_WIDTH, CHECKERBOARD_HEIGHT),
            flags=cv.CALIB_CB_FAST_CHECK
        )


        if (ret == True):
            detected+=1
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), TERMINATION_CRITERIA)
            imgpoints.append(corners2)
            #cv.drawChessboardCorners(frame, (CHECKERBOARD_WIDTH, CHECKERBOARD_HEIGHT), corners2, ret)
        else:
            pass
    else:
        skip_counter-=1

objpoints = np.array(objpoints)
imgpoints = np.array(imgpoints)
    
print('Recall', detected / inspected)
    
cap.release()
cv.destroyAllWindows()

Recall 0.9863013698630136


### 2. Recover intrinsic and distortion matrices

We then recover the intrinsic and per-frame extrinsic parameters. We are only interested on the intrinsic ones

In [20]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, camera_matrix, None, None, flags = cv.CALIB_USE_LU)

### 3. Re-projection error

In order to see how good our calibration is, we can calculate the absolute norm between the points projected using the recovered parameters and the ones found by the corner finding algorithm. To find the average error, we calculate the arithmetical mean of the errors calculated for all the calibration images

In [21]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "Total error: {}".format(mean_error/len(objpoints)))

total error: 0.07953585767808856


### 4. Save the results

In [ ]:
np.save(CAM_MATRIX_PATH, mtx)
np.save(DISTORTION_PATH, dist)